Task 3

Using Python convert data from task2 into  Parquet and save as a copy (For example for Java check https://hadoop.apache.org/docs/r2.7.5/api/org/apache/hadoop/fs/FileSystem.html ). 

Copy scripts to the archive. Make screenshot of parquet contents.

While uploading large files directly in HDFS via standard tools you'll get some problems due to file size. The workaround is to use batch loading (requires additional scripts in Python/Java/Scala)

https://towardsdatascience.com/why-and-how-to-use-pandas-with-large-data-9594dda2ea4c

EXPECTED OUTPUTS:
·         ZIP-ed folder with your screenshots


In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd

Make list of used files

In [2]:
root = 'hdfs://172.17.0.3/user/hive/warehouse'

files = ('destinations',
          'sample_submission',
          'test',
          'train')

csvs = [f'{root}/{file}/{file}.csv' for file in files]

csvs              

['hdfs://172.17.0.3/user/hive/warehouse/destinations/destinations.csv',
 'hdfs://172.17.0.3/user/hive/warehouse/sample_submission/sample_submission.csv',
 'hdfs://172.17.0.3/user/hive/warehouse/test/test.csv',
 'hdfs://172.17.0.3/user/hive/warehouse/train/train.csv']

Start Spark session

In [3]:
sparkSession = SparkSession.builder.appName("read-csv-write-parquet").getOrCreate()

21/08/05 19:56:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Define function for reading data

In [4]:
def read_csv_from_hdfs(path):
    """Reads csv from HDFS as Sparf DF"""
    df_load = sparkSession.read.csv(path, header = 'true', inferSchema = True)
    return(df_load)

In [5]:
#optionally check reading inside
read_csv_from_hdfs(csvs[0]).head()

21/08/05 19:57:01 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(srch_destination_id=0, d1=-2.19865708695, d2=-2.19865708695, d3=-2.19865708695, d4=-2.19865708695, d5=-2.19865708695, d6=-1.89762709129, d7=-2.19865708695, d8=-2.19865708695, d9=-1.89762709129, d10=-2.19865708695, d11=-2.19865708695, d12=-2.19865708695, d13=-2.19865708695, d14=-2.19865708695, d15=-2.19865708695, d16=-1.89762709129, d17=-2.19865708695, d18=-2.19865708695, d19=-2.19865708695, d20=-2.19865708695, d21=-2.19865708695, d22=-2.19865708695, d23=-2.19865708695, d24=-2.19865708695, d25=-2.19865708695, d26=-2.19865708695, d27=-2.19865708695, d28=-2.19865708695, d29=-2.19865708695, d30=-2.19865708695, d31=-2.19865708695, d32=-2.19865708695, d33=-2.19865708695, d34=-2.19865708695, d35=-2.19865708695, d36=-2.19865708695, d37=-1.89762709129, d38=-2.19865708695, d39=-2.19865708695, d40=-2.19865708695, d41=-2.19865708695, d42=-2.19865708695, d43=-1.89762709129, d44=-2.19865708695, d45=-2.19865708695, d46=-2.19865708695, d47=-2.19865708695, d48=-2.19865708695, d49=-2.19865708695, d5

In [6]:
def write_spark_df_to_parquet(spark_df, location):
    return spark_df.write.parquet(location + '.parquet')

In [7]:
for file in files:
    in_location = f'{root}/{file}/{file}.csv'
    out_location = f'staging/{file}'
    print('reading', in_location)
    write_spark_df_to_parquet(read_csv_from_hdfs(in_location), out_location)
    print(f'writed to /{out_location}.parquet')

reading hdfs://172.17.0.3/user/hive/warehouse/destinations/destinations.csv


writed to /staging/destinations.parquet
reading hdfs://172.17.0.3/user/hive/warehouse/sample_submission/sample_submission.csv


writed to /staging/sample_submission.parquet
reading hdfs://172.17.0.3/user/hive/warehouse/test/test.csv


writed to /staging/test.parquet
reading hdfs://172.17.0.3/user/hive/warehouse/train/train.csv


writed to /staging/train.parquet


In [8]:
!ls  staging/*

staging/destinations.parquet:
part-00000-738fdd21-f3b9-418b-94c8-3cdacc43d476-c000.snappy.parquet
part-00001-738fdd21-f3b9-418b-94c8-3cdacc43d476-c000.snappy.parquet
part-00002-738fdd21-f3b9-418b-94c8-3cdacc43d476-c000.snappy.parquet
part-00003-738fdd21-f3b9-418b-94c8-3cdacc43d476-c000.snappy.parquet
_SUCCESS

staging/sample_submission.parquet:
part-00000-b0a06951-9918-4641-9f82-29b7fb7fe161-c000.snappy.parquet
part-00001-b0a06951-9918-4641-9f82-29b7fb7fe161-c000.snappy.parquet
part-00002-b0a06951-9918-4641-9f82-29b7fb7fe161-c000.snappy.parquet
part-00003-b0a06951-9918-4641-9f82-29b7fb7fe161-c000.snappy.parquet
_SUCCESS

staging/test.parquet:
part-00000-6152afce-b7cb-4697-af06-7839bfcd7535-c000.snappy.parquet
part-00001-6152afce-b7cb-4697-af06-7839bfcd7535-c000.snappy.parquet
part-00002-6152afce-b7cb-4697-af06-7839bfcd7535-c000.snappy.parquet
part-00003-6152afce-b7cb-4697-af06-7839bfcd7535-c000.snappy.parquet
_SUCCESS

staging/train.parquet:
part-00000-06e44650-1794-4ac9-8976-5af8b0914

In [9]:
!echo ================== head of binary parquet =========================
!head staging/train.parquet/part-00030-f3f5a0fc-84f2-4ac3-9088-4132f411faab-c000.snappy.parquet
!echo ================== tail of binary parquet =========================
!tail staging/train.parquet/part-00030-f3f5a0fc-84f2-4ac3-9088-4132f411faab-c000.snappy.parquet

================== head of binary parquet =========================
head: cannot open 'staging/train.parquet/part-00030-f3f5a0fc-84f2-4ac3-9088-4132f411faab-c000.snappy.parquet' for reading: No such file or directory
================== tail of binary parquet =========================
tail: cannot open 'staging/train.parquet/part-00030-f3f5a0fc-84f2-4ac3-9088-4132f411faab-c000.snappy.parquet' for reading: No such file or directory


Show content in pandas dataframe 

In [10]:
pd.read_parquet('staging/destinations.parquet')

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690
2,2,-2.183490,-2.224164,-2.224164,-2.189562,-2.105819,-2.075407,-2.224164,-2.118483,-2.140393,...,-2.224164,-2.224164,-2.196379,-2.224164,-2.192009,-2.224164,-2.224164,-2.224164,-2.224164,-2.057548
3,3,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.115485,-2.177409,-2.177409,-2.177409,...,-2.161081,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409
4,4,-2.189562,-2.187783,-2.194008,-2.171153,-2.152303,-2.056618,-2.194008,-2.194008,-2.145911,...,-2.187356,-2.194008,-2.191779,-2.194008,-2.194008,-2.185161,-2.194008,-2.194008,-2.194008,-2.188037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62101,64989,-2.145876,-2.158016,-2.216621,-2.158016,-2.006209,-2.102505,-2.216621,-2.157203,-2.216621,...,-2.216621,-2.158016,-2.158016,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.152024
62102,64990,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.176802,-2.203479,-2.203479,-2.203479,...,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.008885
62103,64991,-2.190567,-2.208308,-2.208308,-2.208308,-2.208308,-2.135493,-2.208308,-2.208308,-1.909808,...,-2.188935,-2.208308,-2.175594,-2.208308,-2.208308,-2.199347,-2.208308,-2.208308,-2.208308,-2.208308
62104,64992,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,...,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686
